In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sqlalchemy import text
import os
import re
import json
import requests



## Load the data

In [2]:
train_files = [
    "final/2024-05-20/2024-05-20_07.json",
    "final/2024-05-20/2024-05-20_08.json",

]

test_files = [
    "final/2024-05-20/teste-2024-05-20_09.json",
]

answer_files = [
    "validation/2024-05-13/resposta-2024-05-13_13.json"
]
train1 = pd.read_json(train_files[0], encoding='latin-1')
train2 = pd.read_json(train_files[1], encoding='latin-1')

train = pd.concat([train1, train2])
test = pd.read_json(test_files[0], encoding='latin-1')

## Filter data

In [3]:
train['latitude'] = train['latitude'].str.replace(',', '.').astype(float)
train['longitude'] = train['longitude'].str.replace(',', '.').astype(float)
train['linha'] = train['linha'].astype(str)

valid_linhas = [
    '483', '864', '639', '3', '309', '774', '629', '371', '397', '100', '838', 
    '315', '624', '388', '918', '665', '328', '497', '878', '355', '138', '606', 
    '457', '550', '803', '917', '638', '2336', '399', '298', '867', '553', '565', 
    '422', '756', '186012003', '292', '554', '634', '232', '415', '2803', '324', 
    '852', '557', '759', '343', '779', '905', '108'
]

df_train = train[train['linha'].isin(valid_linhas)]

## Create dataframe with last 2 points of each line and bus order

In [4]:
df_last_two = df_train.groupby(['ordem', 'linha']).tail(2).reset_index(drop=True)

counts = df_last_two.groupby(['ordem', 'linha']).size()
to_duplicate = counts[counts == 1].index

duplicated_rows = df_last_two.set_index(['ordem', 'linha']).loc[to_duplicate].reset_index()
df_last_two = pd.concat([df_last_two, duplicated_rows]).sort_values(['ordem', 'linha'])


df_last_two = df_last_two[['ordem','linha','latitude','longitude','datahoraservidor']]

## Join data to predict with last two points dataframe


In [5]:
test['linha'] = test['linha'].astype(str)

df_test = test[test['linha'].isin(valid_linhas)]

# Join the two dataframes
join_df = pd.merge(df_test, df_last_two, on=['ordem','linha'], how='inner')
join_df

,id,linha,ordem,datahora,latitude,longitude,datahoraservidor
0,2384594492,2336,D87713,1716203184000,-22.88934,-43.55981,1716202765000
1,2384594492,2336,D87713,1716203184000,-22.88927,-43.55983,1716202796000
2,10300767826,553,C47891,1716203604000,-22.98277,-43.22356,1716202779000
3,10300767826,553,C47891,1716203604000,-22.98098,-43.22396,1716202780000
4,14139244023,864,D86161,1716204857000,-22.90167,-43.55602,1716202789000
...,...,...,...,...,...,...,...
323037,999963343402356,415,A48048,1716203185000,-22.93200,-43.23992,1716202800000
323038,999976895779326,606,B25583,1716203097000,-22.90662,-43.27637,1716202602000
323039,999976895779326,606,B25583,1716203097000,-22.90757,-43.27717,1716202643000
323040,999999304279971,324,B28568,1716204112000,-22.85508,-43.24744,1716202788000


## Model - Predict the next position of the bus based on past positions

In [6]:
database_url = os.getenv("DATABASE_URL")
engine = create_engine(database_url)  

In [7]:
def execute_query(connection, linha, lat1, lon1, lat2, lon2, last_date, prediction_date):
    query = """
    WITH initial_similar_points AS (
        SELECT time_ranking,
               ordem,
               linha,
               x,
               y,
               datahoraservidor
        FROM vw_buses_order
        WHERE linha = :linha
        AND x = width_bucket(:lon1, -43.726090, -42.951470, 1587)
        AND y = width_bucket(:lat1, -23.170790, -22.546410, 1389)
        AND (
                (datahoraservidor >= TO_TIMESTAMP(:last_date) - interval '7 day' - interval '2 hour'  
                AND datahoraservidor < TO_TIMESTAMP(:last_date) - interval '7 day' + interval '2 hour') 
                OR 
                (datahoraservidor >= TO_TIMESTAMP(:last_date) - interval '14 day' - interval '2 hour'  
                AND datahoraservidor < TO_TIMESTAMP(:last_date) - interval '14 day' + interval '2 hour')
                OR 
                (datahoraservidor >= TO_TIMESTAMP(:last_date) - interval '21 day' - interval '2 hour'  
                AND datahoraservidor < TO_TIMESTAMP(:last_date) - interval '21 day' + interval '2 hour')
            )
        AND time_ranking > 1
        LIMIT 10
    ), anterior_points AS (
        SELECT DISTINCT ON (time_ranking, ordem, linha) 
            time_ranking,
            ordem,
            linha,
            x,
            y,
            datahoraservidor
        FROM vw_buses_order
        WHERE (ordem, linha, time_ranking) IN (
            SELECT ordem, linha, time_ranking - 1
            FROM initial_similar_points
            )
    ), direction_points AS (
         SELECT 
            sp.ordem,
            sp.datahoraservidor
        FROM initial_similar_points sp
        INNER JOIN anterior_points ap
            ON sp.ordem = ap.ordem
            AND sp.linha = ap.linha
            AND sp.time_ranking = ap.time_ranking + 1
        WHERE ((ap.x - sp.x) * (:lon2 - :lon1) + (ap.y - sp.y) * (:lat2 - :lat1)) >= 0
    ), first_future_points AS (
        SELECT DISTINCT ON (vo.ordem, vo.datahoraservidor)
            vo.x,
            vo.y,
            vo.ordem,
            vo.datahoraservidor              
        FROM (
                SELECT 
                          ordem,
                          linha,
                          x,
                          y,
                          datahoraservidor
                FROM vw_buses_order
                WHERE linha = :linha
                AND ordem IN (SELECT DISTINCT ordem FROM direction_points)
             ) vo
        INNER JOIN direction_points dp
            ON vo.ordem = dp.ordem
            AND vo.datahoraservidor > dp.datahoraservidor
            AND vo.datahoraservidor < dp.datahoraservidor + interval '1 hour' + interval '20 minutes'
        WHERE vo.datahoraservidor > dp.datahoraservidor + (TO_TIMESTAMP(:prediction_date) - TO_TIMESTAMP(:last_date) - interval '2 minutes')
        AND vo.datahoraservidor < dp.datahoraservidor + (TO_TIMESTAMP(:prediction_date) - TO_TIMESTAMP(:last_date) + interval '2 minutes')
    ), selected_future_points AS (
        SELECT x,y
        FROM first_future_points
    )
    SELECT 
        ROUND(PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY x)) AS median_x,
        ROUND(PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY y)) AS median_y
    FROM selected_future_points;
    """
    
    params = {
        'linha': linha,
        'lat1': lat1,
        'lon1': lon1,
        'lat2': lat2,
        'lon2': lon2,
        'last_date': str(last_date),
        'prediction_date': str(prediction_date)
    }
    
    
    result = connection.execute(text(query), params)
    row = result.fetchone()
        
    return row[0], row[1]

In [8]:
median_x_list = []
median_y_list = []

with engine.connect() as connection:
    for i in range(0, len(join_df)- 1, 2):
        row1 = join_df.iloc[i + 1]
        row2 = join_df.iloc[i]
        median_x, median_y = execute_query(
            connection,
            row1['linha'], 
            float(row1['latitude']), 
            float(row1['longitude']), 
            float(row2['latitude']), 
            float(row2['longitude']), 
            row1['datahoraservidor']/1000, # Convert to seconds - Last Date
            row1['datahora']/1000 # Convert to seconds - Prediction Date
        )

        median_x_list.extend([median_x, median_x])
        median_y_list.extend([median_y, median_y])

join_df['median_x'] = median_x_list
join_df['median_y'] = median_y_list

df_prediction = join_df[['id','latitude', 'longitude', 'median_y','median_x']]

### Count nulls (%)

In [9]:
(df_prediction.isnull().sum()/len(df_prediction)) * 100

id             0.0
latitude       0.0
longitude      0.0
median_y     100.0
median_x     100.0
dtype: float64

### Convert the prediction to latitude and longitude

In [10]:
def inverse_width_bucket_x(bucket_index):
    if bucket_index is None: # Handle None input
        return None
    min_value = -43.726090
    max_value = -42.951470
    num_buckets = 1587

    if bucket_index < 1:
        return min_value
    elif bucket_index > num_buckets:
        return max_value
    
    bucket_size = (max_value - min_value) / num_buckets
    value = min_value + (bucket_index - 1) * bucket_size + bucket_size / 2
    
    return value
    
def inverse_width_bucket_y(bucket_index):
    if bucket_index is None: # Handle None input
        return None
    min_value = -23.170790
    max_value = -22.546410
    num_buckets = 1389

    if bucket_index < 1:
        return min_value
    elif bucket_index > num_buckets:
        return max_value
    
    bucket_size = (max_value - min_value) / num_buckets
    value = min_value + (bucket_index - 1) * bucket_size + bucket_size / 2
    
    return value

df_prediction['median_x'] = df_prediction['median_x'].apply(inverse_width_bucket_x)
df_prediction['median_y'] = df_prediction['median_y'].apply(inverse_width_bucket_y)

C:\Users\Lucas S. Tavares\AppData\Local\Temp\ipykernel_4472\1796196836.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_prediction['median_x'] = df_prediction['median_x'].apply(inverse_width_bucket_x)
C:\Users\Lucas S. Tavares\AppData\Local\Temp\ipykernel_4472\1796196836.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_prediction['median_y'] = df_prediction['median_y'].apply(inverse_width_bucket_y)


### Fill Na values with latitude and longitude columns


In [11]:
# Keep only newest rows for null values
even_indices = df_prediction.index[df_prediction.index % 2 == 0]
rows_with_nulls = df_prediction.loc[even_indices].isnull().any(axis=1)
df_prediction = df_prediction.drop(even_indices[rows_with_nulls])


In [12]:
df_prediction['median_x'] = df_prediction['median_x'].fillna(df_prediction['longitude'])
df_prediction['median_y'] = df_prediction['median_y'].fillna(df_prediction['latitude'])

C:\Users\Lucas S. Tavares\AppData\Local\Temp\ipykernel_4472\1504000778.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_prediction['median_x'] = df_prediction['median_x'].fillna(df_prediction['longitude'])
C:\Users\Lucas S. Tavares\AppData\Local\Temp\ipykernel_4472\1504000778.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_prediction['median_y'] = df_prediction['median_y'].fillna(df_prediction['latitude'])


In [13]:
df_prediction = df_prediction[['id','median_y','median_x']]

In [14]:
df_prediction.drop_duplicates(inplace=True)

## Criar json com resposta

In [15]:
test_files[0][25:36]

'24-05-20_09'

In [16]:
df_prediction['id'] = df_prediction['id'].astype('Int64')
previsoes = df_prediction.values.tolist()

match = re.search(r'teste-(\d{4}-\d{2}-\d{2})_(\d{2})', test_files[0])
date_part = match.group(1)
hour_part = match.group(2)
datahora = f"{date_part} {hour_part}:00:00"

output = {
    "aluno": "Lucas Tavares Da Silva Ferreira",
    "datahora": datahora,
    "previsoes": [[str(item) if isinstance(item, pd.Int64Dtype) else item for item in row] for row in previsoes],
    "senha": "SfC-/CM5wefsN62"
}

output_json = json.dumps(output, indent=4)

output_filename = test_files[0][25:36] + "_answer.json"
with open(output_filename, "w") as json_file:
    json_file.write(output_json)

print(f"JSON salvo em {output_filename}")

JSON salvo em 24-05-20_09_answer.json


### Fazer a requisição POST

In [17]:
# Fazer o POST usando a biblioteca requests
url = 'https://barra.cos.ufrj.br:443/datamining/rpc/avalia'
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json'
}

In [18]:
response = requests.post(url, headers=headers, data=output_json)

# Verificar a resposta
if response.status_code == 200:
    print("POST bem-sucedido!")
    print("Resposta do servidor:")
    print(response.json())
else:
    print(f"Falha no POST: {response.status_code}")
    print(response.text)

POST bem-sucedido!
Resposta do servidor:
{'msg': 'Problemas!', 'arquivo teste': 'teste-2024-05-20_09.json', 'rmse': 6897.804584229733, 'ids não encontrados': 0, 'ids testados': 161521, 'total na tabela': 164537}


## Compare the distance between predicted and real values

In [19]:
answer = pd.read_json(answer_files[0], encoding='latin-1')
answer['latitude'] = answer['latitude'].str.replace(',', '.').astype(float)
answer['longitude'] = answer['longitude'].str.replace(',', '.').astype(float)
answer

,id,latitude,longitude
0,1,-22.88206,-43.32547
1,2,-22.90037,-43.23099
2,3,-22.88279,-43.29586
3,4,-22.90811,-43.23055
4,5,-22.88593,-43.30216
...,...,...,...
156637,156638,-22.87820,-43.24119
156638,156639,-22.83674,-43.28420
156639,156640,-22.96980,-43.18537
156640,156641,-22.84024,-43.27761


In [20]:
# Function to calculate the distance between two points in a sphere in meters
def haversine(lat1, lon1, lat2, lon2):
    R = 6371000  # radius of Earth in meters
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    
    a = np.sin(delta_phi / 2.0) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2.0) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    
    meters = R * c
    return meters

final_df = answer.merge(df_prediction, on='id', suffixes=('_true', '_pred'))

final_df['error_meters'] = final_df.apply(
    lambda row: haversine(row['latitude'], row['longitude'], row['median_y'], row['median_x']), axis=1
)



## Statistics

In [21]:
# Ensure pandas is imported as pd in a previous cell or at the beginning of this cell.
# For example: import pandas as pd

# Check if 'final_df' and 'error_meters' column exist
if 'final_df' in locals() and isinstance(final_df, pd.DataFrame) and 'error_meters' in final_df.columns:
    # Attempt to convert 'error_meters' to a numeric type, coercing errors to NaN
    # This helps if the column is of object type or contains non-numeric strings.
    final_df['error_meters'] = pd.to_numeric(final_df['error_meters'], errors='coerce')

    # Diagnostic prints to understand the state of 'error_meters'
    print("--- Diagnostics for 'error_meters' column ---")
    print(f"Data type: {final_df['error_meters'].dtype}")
    nan_count = final_df['error_meters'].isnull().sum()
    total_count = len(final_df['error_meters'])
    print(f"NaN values: {nan_count} / {total_count} total entries")
    
    if total_count > 0:
        if nan_count == total_count:
            print("Warning: Column is entirely NaN values after coercion.")
        else:
            print(f"Non-NaN values: {final_df['error_meters'].notnull().sum()}")
            print("First 5 non-NaN values (if any):")
            print(final_df['error_meters'].dropna().head())
    elif total_count == 0:
        print("Warning: Column is empty.")
    print("---------------------------------------------")

    # Calculate statistics (pandas functions skip NaN by default)
    mean_error = final_df['error_meters'].mean()
    std_deviation = final_df['error_meters'].std()
    median_error = final_df['error_meters'].median()

    print(f'Average error in meters: {mean_error:.2f} m')
    print(f'Standard deviation of the error in meters: {std_deviation:.2f} m')
    print(f'Median error in meters: {median_error:.2f} m')

    # Additional check if statistics are still NaN
    if pd.isna(mean_error) and total_count > 0 and nan_count < total_count:
        print("\nNote: Statistics are NaN. This can happen if all non-NaN values are problematic for the calculation, or if an underlying issue persists.")
    elif pd.isna(mean_error) and (total_count == 0 or (total_count > 0 and nan_count == total_count)):
         print("\nNote: Statistics are NaN because the 'error_meters' column is empty or contains only NaN values after processing.")

elif 'final_df' in locals() and isinstance(final_df, pd.DataFrame) and 'error_meters' not in final_df.columns:
    print("Error: 'error_meters' column not found in final_df.")
    mean_error, std_deviation, median_error = float('nan'), float('nan'), float('nan')
    print(f'Average error in meters: {mean_error:.2f} m')
    print(f'Standard deviation of the error in meters: {std_deviation:.2f} m')
    print(f'Median error in meters: {median_error:.2f} m')
else:
    print("Error: DataFrame 'final_df' not found or is not a DataFrame. Please ensure it is loaded correctly.")
    mean_error, std_deviation, median_error = float('nan'), float('nan'), float('nan')
    print(f'Average error in meters: {mean_error:.2f} m')
    print(f'Standard deviation of the error in meters: {std_deviation:.2f} m')
    print(f'Median error in meters: {median_error:.2f} m')

--- Diagnostics for 'error_meters' column ---
Data type: float64
NaN values: 0 / 0 total entries
---------------------------------------------
Average error in meters: nan m
Standard deviation of the error in meters: nan m
Median error in meters: nan m

Note: Statistics are NaN because the 'error_meters' column is empty or contains only NaN values after processing.
